In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# print(pd.__version__)
import leaguedata as ld

#Progress bars wooo
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import Markdown

In [2]:
matches = ld.get_all_matches()
print(len(matches))

7620


In [59]:
num_games = 50
data = None
# prev_structures = test.prev_kills(time)
# prev_structures_5_mins = prev_structures.loc[prev_structures['Time'] > (time-5)]
# print(prev_structures)
# print(prev_structures_5_mins)
# print(len(prev_structures_5_mins.index)/5)
# print(test.kill_freq(time))
# print(test.prev_kills(20)[2])
# print(test.game_state(5))

for num in range(num_games):
    game = matches[num]
    for minute in range(1,game.length()):
        segment_data = game.game_state(minute)
#         print(segment_data)
        if minute == 0 and num == 0:
            data = segment_data
        else:
            data = pd.concat([data,segment_data],ignore_index=True)
print(data)

      minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0          1           0            0        0.0             0.0   
1          2           0            0        0.0             0.0   
2          3           0            0        0.0             0.0   
3          4           0            0        0.0             0.0   
4          5           0            0        0.0             0.0   
...      ...         ...          ...        ...             ...   
1873      25           8            3        0.2             0.2   
1874      26           8            3        0.2             0.2   
1875      27          11            5        1.2             0.8   
1876      28          11            5        1.2             0.8   
1877      29          13            5        1.4             1.0   

      Blue death freq  Red structures taken  Blue structures lost  \
0                 0.0                     0                     0   
1                 0.0                     0  

In [71]:
#Standardisation for LASSO
datatest = data
means = data.mean(axis=0)
stds = data.std(axis=0)
#print(data)
datatest = (datatest - means) / stds
datatest["Blue Win"] = data["Blue Win"] # set the wins back to bool
print(datatest)

        minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0    -1.599766   -0.910775    -0.863774  -0.969282       -0.812669   
1    -1.514938   -0.910775    -0.863774  -0.969282       -0.812669   
2    -1.430110   -0.910775    -0.863774  -0.969282       -0.812669   
3    -1.345282   -0.910775    -0.863774  -0.969282       -0.812669   
4    -1.260453   -0.910775    -0.863774  -0.969282       -0.812669   
...        ...         ...          ...        ...             ...   
1873  0.436111    0.604561    -0.248073  -0.626280       -0.269733   
1874  0.520939    0.604561    -0.248073  -0.626280       -0.269733   
1875  0.605767    1.172812     0.162395   1.088730        1.359074   
1876  0.690596    1.172812     0.162395   1.088730        1.359074   
1877  0.775424    1.551646     0.162395   1.431731        1.902009   

      Blue death freq  Red structures taken  Blue structures lost  \
0           -0.797540             -0.887736             -0.863601   
1           -0.797540

In [72]:
#This step makes it real u feel me? only run once or ur original data from the 20 minute step previously is lost
data_backup = data
data = datatest

In [73]:
Penalty = 1 #Penalty: 1 small - 0.01 big, from slides
from sklearn.linear_model import LogisticRegression as logreg
# print(titanic[["Pclass_1","Pclass_2","Pclass_3"]])

y = data['Blue Win'].values#.reshape(-1, 1)
print(y)
X = data.drop('Blue Win', axis=1)

# print(X)
# print(y)
mylr = logreg(C=Penalty, penalty='l1', solver='liblinear')
mylr.fit(X,y)

[ True  True  True ...  True  True  True]


LogisticRegression(C=1, penalty='l1', solver='liblinear')

In [74]:
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

class ModelSummary:
    """ This class extracts a summary of the model
    
    Methods
    -------
    get_se()
        computes standard error
    get_ci(SE_est)
        computes confidence intervals
    get_pvals()
        computes p-values
    get_summary(name=None)
        prints the summary of the model
    """
    
    def __init__(self, clf, X, y):
        """
        Parameters
        ----------
        clf: class
            the classifier object model
        X: pandas Dataframe
            matrix of predictors
        y: numpy array
            matrix of variable
        """
        self.clf = clf
        self.X = X
        self.y = y
        pass
    
    def get_se(self):
        """Computes the standard error

        Returns
        -------
            numpy array of standard errors
        """
        # from here https://stats.stackexchange.com/questions/89484/how-to-compute-the-standard-errors-of-a-logistic-regressions-coefficients
        predProbs = self.clf.predict_proba(self.X)
        X_design = np.hstack([np.ones((self.X.shape[0], 1)), self.X])
        V = np.diagflat(np.product(predProbs, axis=1))
        covLogit = np.linalg.inv(np.dot(np.dot(X_design.T, V), X_design))
        return np.sqrt(np.diag(covLogit))

    def get_ci(self, SE_est):
        """Computes the confidence interval

        Parameters
        ----------
        SE_est: numpy array
            matrix of standard error estimations
        
        Returns
        -------
        cis: numpy array
            matrix of confidence intervals
        """
        p = 0.975
        df = len(self.X) - 2
        crit_t_value = stats.t.ppf(p, df)
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        upper = coefs + (crit_t_value * SE_est)
        lower = coefs - (crit_t_value * SE_est)
        cis = np.zeros((len(coefs), 2))
        cis[:,0] = lower
        cis[:,1] = upper
        return cis
    
    def get_pvals(self):
        """Computes the p-value

        Returns
        -------
        p: numpy array
            matrix of p-values
        """
        # from here https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance
        p = self.clf.predict_proba(self.X)
        n = len(p)
        m = len(self.clf.coef_[0]) + 1
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        se = self.get_se()
        t =  coefs/se  
        p = (1 - stats.norm.cdf(abs(t))) * 2
        return p
    
    def get_summary(self, names=None):
        """Prints the summary of the model

        Parameters
        ----------
        names: list
            list of the names of predictors
        """
        ses = self.get_se()
        cis = self.get_ci(ses)
        lower = cis[:, 0]
        upper = cis[:, 1]
        pvals = self.get_pvals()
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        data = []
        for i in range(len(coefs)):
            currlist = []
            currlist.append(np.round(coefs[i], 3))
            currlist.append(np.round(ses[i], 3))
            currlist.append(np.round(pvals[i], 3))
            currlist.append(np.round(lower[i], 3))
            currlist.append(np.round(upper[i], 3))
            data.append(currlist)
        cols = ['coefficient', 'std', 'p-value', '[0.025', '0.975]']
        sumdf = pd.DataFrame(columns=cols, data=data)
        if names is not None:
            new_names = ['intercept']*(len(names) + 1)
            new_names[1:] = [i for i in names]
            sumdf.index = new_names
        else:
            try:
                names = list(self.X.columns)
                new_names = ['intercept']*(len(names) + 1)
                new_names[1:] = [i for i in names]
                sumdf.index = new_names
            except:
                pass
        print(sumdf)
        acc = accuracy_score(self.y, self.clf.predict(self.X))
        confmat = confusion_matrix(self.y, self.clf.predict(self.X))
        print('-'*60)
        print('Confusion Matrix (total:{}) \t Accuracy: \t  {}'.format(len(self.X),np.round(acc, 3)))
        print('  TP: {} | FN: {}'.format(confmat[1][1],confmat[1][0]))
        print('  FP: {} | TN: {}'.format(confmat[0][1],confmat[0][0]))

In [12]:
modsummary = ModelSummary(mylr, X, y)
modsummary.get_summary()

                      coefficient           std  p-value        [0.025  \
intercept                   0.505  7.190000e-01    0.482 -9.180000e-01   
minute                     -0.000  1.050000e-01    1.000 -2.080000e-01   
Blue kills                  0.714  8.480000e-01    0.399 -9.650000e-01   
Blue Deaths                -2.528  1.441000e+00    0.079 -5.382000e+00   
Kill freq                   0.609  5.985647e+07    1.000 -1.185865e+08   
Blue kill freq              0.000  5.994801e+07    1.000 -1.187678e+08   
Blue death freq             0.000  5.982868e+07    1.000 -1.185314e+08   
Red structures taken        2.239  7.110000e-01    0.002  8.290000e-01   
Blue structures lost       -1.735  1.177000e+00    0.140 -4.066000e+00   
Structure freq              0.000  1.299803e+08    1.000 -2.575145e+08   
Blue structure freq         0.000  1.299803e+08    1.000 -2.575145e+08   
Red structure freq          0.000  1.299803e+08    1.000 -2.575145e+08   
Gold diff                   0.001  1.0

In [75]:
#testing
import random

# test = matches[12]
# print(random.randint(0,test.length()))
num_test_games = 50
test_data = None
# for num in range(num_games+1,num_test_games+num_games+1):
#     print("test")


for num in range(num_games+1,num_test_games+num_games+1):
    game = matches[num]
    for minute in range(1,game.length()):
        test_segment_data = game.game_state(minute)
#         print(segment_data)
        if minute == 0 and num == num_games+1:
            test_data = test_segment_data
        else:
            test_data = pd.concat([test_data,test_segment_data],ignore_index=True)
# for num in range(num_games+1,num_test_games):
#     game = matches[num]
#     for minute in range(game.length()):
#         test_segment_data = game.game_state(minute)
#         if i == 0 and num == num_games+1:
#             test_data = test_segment_data
#         else:
#             test_data = pd.concat([test_data,test_segment_data],ignore_index=True)
        
print(test_data)
# print(data)

      minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0          1           0            0        0.0             0.0   
1          2           0            0        0.0             0.0   
2          3           0            0        0.0             0.0   
3          4           0            0        0.0             0.0   
4          5           0            0        0.0             0.0   
...      ...         ...          ...        ...             ...   
1877      30           9            8        0.6             0.4   
1878      31           9            8        0.6             0.4   
1879      32          14           10        2.0             1.4   
1880      33          14           10        1.6             1.2   
1881      34          14           10        1.4             1.0   

      Blue death freq  Red structures taken  Blue structures lost  \
0                 0.0                     0                     0   
1                 0.0                     0  

In [76]:
test_X = test_data.drop('Blue Win', axis=1)
real_y = test_data['Blue Win'].values.reshape(-1, 1)

print(test_X)
print(real_y)


      minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0          1           0            0        0.0             0.0   
1          2           0            0        0.0             0.0   
2          3           0            0        0.0             0.0   
3          4           0            0        0.0             0.0   
4          5           0            0        0.0             0.0   
...      ...         ...          ...        ...             ...   
1877      30           9            8        0.6             0.4   
1878      31           9            8        0.6             0.4   
1879      32          14           10        2.0             1.4   
1880      33          14           10        1.6             1.2   
1881      34          14           10        1.4             1.0   

      Blue death freq  Red structures taken  Blue structures lost  \
0                 0.0                     0                     0   
1                 0.0                     0  

In [77]:
y_predict = mylr.predict(test_X)
print(y_predict)

[False False False ...  True  True  True]


In [78]:
test_summary = ModelSummary(mylr, test_X, real_y)
test_summary.get_summary()

                      coefficient           std  p-value        [0.025  \
intercept                   0.477  5.920000e-01    0.421 -6.840000e-01   
minute                     -0.985  3.880000e-01    0.011 -1.746000e+00   
Blue kills                  2.903  5.914000e+00    0.624 -8.696000e+00   
Blue Deaths                -3.360  3.856510e+02    0.993 -7.597090e+02   
Kill freq                   0.000           NaN      NaN           NaN   
Blue kill freq             -0.071           NaN      NaN           NaN   
Blue death freq             0.017           NaN      NaN           NaN   
Red structures taken        0.549  9.119230e+02    1.000 -1.787938e+03   
Blue structures lost        1.061  1.510141e+03    0.999 -2.960669e+03   
Structure freq              0.000  4.952396e+08    1.000 -9.712770e+08   
Blue structure freq         0.249  4.952396e+08    1.000 -9.712770e+08   
Red structure freq         -0.374  4.952396e+08    1.000 -9.712770e+08   
Gold diff                   1.470  5.8

<ipython-input-74-336093257511>:49: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(covLogit))
<ipython-input-74-336093257511>:49: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(covLogit))
/Users/Sean/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


In [79]:
# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_X,real_y)))
score = 0
for result in range(len(real_y)):
    if real_y[result] == y_predict[result]:
        score += 1
accuracy = score/len(real_y)
print(accuracy)

0.7444208289054197
